# Project Prototype: Agentic RAG System with Nursing Handbooks and Transes as Knowledge Base

The goal of this project is to create an **Agentic RAG-based system** that helps nursing students retrieve relevant information from nursing handbooks and their personal study notes. This system will augment the responses with context from the personal notes, making it more personalized and adaptive to the user's learning.

## Flow Overview

The project involves two primary stages:
1. **Prepopulating the Vector DB** (embedding the nursing handbooks and personal notes into a database)
2. **RAG Modeling** (retrieving relevant information and augmenting responses using both the handbooks and personal notes)

Additionally, there will be an **Agentic Layer** that intelligently routes queries to the appropriate source (nursing handbooks or personal study notes).

### Technologies to use

- **Docling**: For converting nursing handbooks and personal notes into AI-friendly markdown format.
- **OpenAI Embedding Model (large)**: For embedding both nursing handbooks and personal study notes.
- **ChromaDB**: For storing and querying the embeddings.
- **Deepseek LLM**: For augmenting responses based on the retrieved content.
- **Pydantic AI**: For AI agent intelligently routing queries between the nursing handbooks and personal notes. 
- **Pydantic**: For type safety and data validation.
- **FastAPI**: For building the API.
- **Docker**: For containerization and deployment.
- **Pydantic Graphs**: For workflow pipelines

## Initialization

In [48]:
import os
from docling.document_converter import DocumentConverter

knowledge_base_raw_path = './data/knowledge-base/raw/'
knowledge_base_converted_path = './data/knowledge-base/converted/'

### Stage 1: Prepopulate the Vector DB

In this stage, we process nursing handbooks and personal study notes, embedding them into a vector database for efficient retrieval during RAG modeling.

#### Steps:

1. **Upload Nursing Handbooks**:
   - Upload nursing handbooks in popular formats like PDF or DOCX.
   - These documents may include textbooks on topics like **medical-surgical nursing, pharmacology, pediatric nursing**, and other specialized nursing areas.

2. **Convert Documents into AI-Friendly Markdown Format**:
   - Use **Docling** to convert nursing handbooks and personal notes into markdown format.
   - The conversion ensures that headings, bullet points, tables, and other relevant formatting are preserved, making it easier for the AI to process.

3. **Generate Embeddings**:
   - Use **OpenAI's large embedding model** to convert the nursing handbooks and personal notes into embeddings. These embeddings will capture the semantic meaning of each section, allowing for efficient similarity-based searches.
   - Both the nursing handbooks and the personal study notes will be embedded into the vector database.

4. **Save to Vector DB (ChromaDB)**:
   - Store the generated embeddings in **ChromaDB** for fast retrieval during RAG modeling.
   - The vector database will allow the system to quickly access the most relevant information when a query is made.

**Note**: Converting and embedding long documents like nursing handbooks may take some time (e.g., **15-30 minutes** per document).



In [28]:
# List the raw files to convert
os.listdir(knowledge_base_raw_path)

['Handbook of Clinical Nursing_ Medical-Surgical Nursing -- Joyce Fitzpatrick -- 2018 -- Springer Publishing Company -- 9780826130785 -- 26f2533f396508e653d45e0e76aadc53 -- Anna’s Archive.pdf']

In [53]:
document_converter = DocumentConverter()

knowledge_base_raw_files = os.listdir(knowledge_base_raw_path)
knowledge_base_converted_files = os.listdir(knowledge_base_converted_path)

for file in knowledge_base_raw_files:
	file_path = os.path.join(knowledge_base_raw_path, file)
	file_name = os.path.splitext(file)[0]

	# Check if not file 
	if not os.path.isfile(file_path):
		continue

	# Check if file is already converted
	if f"{file_name}.md" in knowledge_base_converted_files:
		print(f"File `{file[:97]}`... is already converted.")
		continue

	# Convert the file into md format that is LLM friendly
	print(f"Converting file: `{file[:97]}`...")
	conversion_result = document_converter.convert(file_path)
	markdown_text = conversion_result.document.export_to_markdown()

	# Write the converted file to the converted folder
	with open(os.path.join(knowledge_base_converted_path, f"{file_name}.md"), "w") as f:
		f.write(markdown_text)


File `Handbook of Clinical Nursing_ Medical-Surgical Nursing -- Joyce Fitzpatrick -- 2018 -- Springer P`... is already converted.


### Stage 2: RAG Modeling

Once the vector database is populated, the system will retrieve relevant information from both nursing handbooks and personal study notes. The retrieval will be augmented using an LLM (Large Language Model) for more accurate and contextually rich responses.

#### Steps:

1. **Retrieve Relevant Information**:
   - When a query is input by the user (e.g., "What are the symptoms of diabetes?"), the system will retrieve the most relevant content from the vector database.
   - ChromaDB will find the sections of the nursing handbooks or personal study notes that are most similar to the query.

2. **Augment Response Using Deepseek LLM**:
   - The retrieved sections will be passed through the **Deepseek LLM**, which will generate a response based on the content of the handbooks and notes.
   - The LLM will combine the relevant information and format it into a coherent, accurate response tailored to the question.

### Stage 3: Agentic RAG System

The **Agentic Layer** adds an intelligent routing mechanism to the RAG system. Instead of simply retrieving data from a static source, the system will decide where to get the most relevant information based on the context of the query and whether the query pertains to the nursing handbooks or her personal study notes.

#### Steps:

1. **Incorporate Personal Study Notes (Transes)**:
   - Personal study notes (transes) are documents or annotations written by the user, capturing key insights, case studies, or specific experiences.
   - These notes will be embedded in the same way as the nursing handbooks and stored in the same vector database.

2. **Agent Routes the Query**:
   - When a user (nursing student) asks a question, the **agent** will decide whether to retrieve data from the nursing handbooks or her personal study notes.
   - The agent will consider:
     - The **context** of the question (general knowledge vs. personalized experiences).
     - Whether her **personal study notes** are more relevant (e.g., if she’s already written notes on a particular case or condition).
     - If the query is general, the agent will prioritize retrieving information from the **nursing handbooks**.

3. **Retrieve from Both Sources**:
   - The system retrieves information from both the **nursing handbooks** and **personal study notes** based on the agent’s decision.
   - If the question pertains to something she has written down in her personal notes, the agent will prioritize retrieving those, combining them with general knowledge from the handbooks.

4. **Augment Response Using Deepseek LLM**:
   - The retrieved content from both sources is fed to the **Deepseek LLM**, which will merge the insights and generate a complete response.
   - The LLM will ensure the answer is not only accurate but also personalized based on the notes, making it more useful and contextually rich.
